In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
df= pd.read_csv("Churn_Modelling.csv")

In [3]:
df.drop(["RowNumber","CustomerId","Surname"],axis=1,inplace=True)

In [4]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

In [5]:
label_ecoder=LabelEncoder()
one_hot_encoder= OneHotEncoder(drop='first')

In [6]:
df["Gender"]= label_ecoder.fit_transform(df["Gender"])

In [7]:
encoding_array= one_hot_encoder.fit_transform(df[["Geography"]]).toarray()

In [8]:
encoding_array

array([[0., 0.],
       [0., 1.],
       [0., 0.],
       ...,
       [0., 0.],
       [1., 0.],
       [0., 0.]])

In [9]:
one_hot_encoder.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]

In [10]:
one_hot_encoder.get_feature_names_out()

array(['Geography_Germany', 'Geography_Spain'], dtype=object)

In [11]:
encoding_df= pd.DataFrame(encoding_array,columns=one_hot_encoder.get_feature_names_out())

In [12]:
label_ecoder.classes_

array(['Female', 'Male'], dtype=object)

In [13]:
df=pd.concat([df,encoding_df],axis=1)

In [14]:
df.drop("Geography",axis=1,inplace=True)

In [15]:
import pickle

In [16]:
with open("one_hot_encoder.pkl","wb") as file:
    pickle.dump(one_hot_encoder,file)

In [17]:
with open("label_ecoder.pkl","wb") as file:
    pickle.dump(label_ecoder,file)

In [18]:
X=df.drop("Exited",axis=1)
y= df["Exited"]

In [19]:
from sklearn.preprocessing import StandardScaler

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train,X_test,y_train,y_test= train_test_split(X,y,random_state=42,shuffle=True,stratify=y,test_size=0.2)

In [22]:
X_train.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_Germany',
       'Geography_Spain'],
      dtype='object')

In [22]:
std= StandardScaler()
X_train= std.fit_transform(X_train)
X_test= std.transform(X_test)

In [24]:
with open("standard_scaler.pkl","wb") as file:
    pickle.dump(std,file)

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [26]:
(X_train.shape[1],)

(11,)

In [27]:
## Build Our ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## HL1 Connected wwith input layer
    Dense(32,activation='relu'), ## HL2
    Dense(1,activation='sigmoid')  ## output layer
]

)

In [28]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             768 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [30]:
## compile the model
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [31]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [32]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [33]:
### Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8187 - loss: 0.4469 - val_accuracy: 0.8545 - val_loss: 0.3620
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8497 - loss: 0.3636 - val_accuracy: 0.8555 - val_loss: 0.3642
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8606 - loss: 0.3419 - val_accuracy: 0.8675 - val_loss: 0.3392
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8580 - loss: 0.3387 - val_accuracy: 0.8635 - val_loss: 0.3442
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8612 - loss: 0.3468 - val_accuracy: 0.8635 - val_loss: 0.3397
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8594 - loss: 0.3347 - val_accuracy: 0.8570 - val_loss: 0.3445
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8593 - loss: 0.3390 - val_accuracy: 0.8555 - val_loss: 0.3556
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8619 - loss: 0.3349 - val_accu

In [34]:
model.save('model.h5')